In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 青海省部分地区
wuhan_ee = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

permanent_water = []

NameError: name 'roi' is not defined

# landsat 8 可用年份为2014-2020

In [ ]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

# 可视化参数设置
vis_params = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

def index(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI")
    return image.addBands(ndvi).addBands(mndwi)

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# 分类标签
label = 'waterclass'

In [ ]:
# 可用年份为2014-2020
Year = 2018
# month = 4
# 定义一些变量
days_31 = [1,3,7,8,10,12]
days_30 = [4,6,9,11]
days_28 = [2]
for year in range(2018,2019,1):
    landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(wuhan_ee) \
        .filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)) \
        .map(maskL8sr) \
        .map(index)
    #合并
    wuhan_landsat8 = landsat_images.qualityMosaic('NDVI').clip(wuhan_ee)
    Map.addLayer(wuhan_landsat8,vis_params,"wuhan")
    # 当mNDWI > EVI时
    wuhan_mNDWI_NDVI = wuhan_landsat8.select('mNDWI').gt(wuhan_landsat8.select('NDVI'))
    wuhan_water = wuhan_mNDWI_NDVI.select('mNDWI').rename('waterclass')

    # Make the training dataset.
    points = wuhan_water.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 1000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    # Overlay the points on the imagery to get training.
    sample_point = wuhan_landsat8.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })

    # 在training要素集中增加一个random属性，值为0到1的随机数
    withRandom = sample_point.randomColumn('random')

    #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
    split = 0.7
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))

    # 利用70%数据随机森林进行训练
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(trainingPartition,label,bands)
    #对Landsat-8进行分类
    result=wuhan_landsat8.select(bands).classify(trainedClassifier)
    Map.addLayer(result,{'min':0,'max':1,'palette': ['white','blue']},'RF_water')

    water = result.eq(1).selfMask().multiply(ee.Image.pixelArea()).divide(1e6)
    areas = water.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': wuhan_ee.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    permanent_water.append(areas.get('classification').getInfo())
    print('{} year:{}'.format(year,areas.get('classification').getInfo()))

# train_accuracy = trainedClassifier.confusionMatrix()
# print('accuracy:',train_accuracy.accuracy().getInfo())
# print('kappa:',train_accuracy.kappa().getInfo())
# print('producersAccuracy:',train_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',train_accuracy.consumersAccuracy().getInfo())

# validated = testingPartition.classify(trainedClassifier)

# test_accuracy = validated.errorMatrix('waterclass', 'classification')

# print('accuracy:',test_accuracy.accuracy().getInfo())
# print('kappa:',test_accuracy.kappa().getInfo())
# print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

# landsat 7 可用年份为1999-2020

In [ ]:
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 7)).Or(qa.bitwiseAnd(1 << 3))
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

# 可视化参数设置
vis_params = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

def NDVI(image):
    ndvi=image.normalizedDifference(['B4', 'B3']).rename('NDVI')  
    return image.addBands(ndvi)

def mNDWI(image):
    mndwi=image.normalizedDifference(['B2', 'B5']).rename("mNDWI")
    return image.addBands(mndwi)

# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
# 分类标签
label = 'waterclass'

In [ ]:
# 可用年份为2010-2020
# year = 2018
# month = 4
# 定义一些变量
days_31 = [1,3,5,7,8,10,12]
days_30 = [4,6,9,11]
days_28 = [2]
for year in range(2010,2014,1):
    landsat_images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
        .filterBounds(wuhan_ee) \
        .filterDate('{}-01-01'.format(year), '{}-12-31'.format(year)) \
        .map(cloudMaskL457) \
        .map(NDVI) \
        .map(mNDWI)
    #合并
    wuhan_landsat7 = landsat_images.qualityMosaic('NDVI').clip(wuhan_ee)
    Map.addLayer(wuhan_landsat7,vis_params,"wuhan")
    # 当mNDWI > EVI时
    wuhan_mNDWI_NDVI = wuhan_landsat7.select('mNDWI').gt(wuhan_landsat7.select('NDVI'))
    wuhan_water = wuhan_mNDWI_NDVI.select('mNDWI').rename('waterclass')

    # Make the training dataset.
    points = wuhan_water.sample(**{
        'region': wuhan_ee,
        'scale': 30,
        'numPixels': 1000,
        'seed': 0,
        'geometries': True  # Set this to False to ignore geometries
    })

    # Overlay the points on the imagery to get training.
    sample_point = wuhan_landsat7.select(bands).sampleRegions(**{
        'collection': points,
        'properties': [label],
        'scale': 30
    })

    # 在training要素集中增加一个random属性，值为0到1的随机数
    withRandom = sample_point.randomColumn('random')

    #把生成随机数大于0.7的分成测试数据，否则为训练数据。这样可以把数据分成两份
    split = 0.7
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))

    # 利用70%数据随机森林进行训练
    trainedClassifier = ee.Classifier.smileRandomForest(20).train(trainingPartition,label,bands)
    #对Landsat-8进行分类
    result=wuhan_landsat7.select(bands).classify(trainedClassifier)
    Map.addLayer(result,{'min':0,'max':1,'palette': ['white','blue']},'RF_water')

    water = result.eq(1).selfMask().multiply(ee.Image.pixelArea()).divide(1e6)
    areas = water.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': wuhan_ee.geometry(),
        'scale': 30,
        'maxPixels': 1e14
    })
    permanent_water.append(areas.get('classification').getInfo())
    print('{} year:{}'.format(year,areas.get('classification').getInfo()))

# train_accuracy = trainedClassifier.confusionMatrix()
# print('accuracy:',train_accuracy.accuracy().getInfo())
# print('kappa:',train_accuracy.kappa().getInfo())
# print('producersAccuracy:',train_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',train_accuracy.consumersAccuracy().getInfo())

# validated = testingPartition.classify(trainedClassifier)

# test_accuracy = validated.errorMatrix('waterclass', 'classification')

# print('accuracy:',test_accuracy.accuracy().getInfo())
# print('kappa:',test_accuracy.kappa().getInfo())
# print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
# print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

# 结果

In [ ]:
print(permanent_water)

使用JRC数据集验证

In [ ]:
JRC_wuhan_water = ee.Image("JRC/GSW1_2/YearlyHistory/{}".format(Year)).clip(wuhan_ee)
JRC_water = JRC_wuhan_water.gt(2).select("waterClass").rename('waterclass')
JRC_points = JRC_water.sample(**{
    'region': wuhan_ee,
    'scale': 30,
    'numPixels': 3000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

# print(points.first().getInfo())
# Overlay the points on the imagery to get training.
JRC_sample_point = wuhan_landsat8.select(bands).sampleRegions(**{
  'collection': JRC_points,
  'properties': [label],
  'scale': 30
})
print(withRandom.first().getInfo())
validated = JRC_sample_point.classify(trainedClassifier)

test_accuracy = validated.errorMatrix('waterclass', 'classification')
print('accuracy:',test_accuracy.accuracy().getInfo())
print('kappa:',test_accuracy.kappa().getInfo())
print('producersAccuracy:',test_accuracy.producersAccuracy().getInfo())
print('consumersAccuracy:',test_accuracy.consumersAccuracy().getInfo())

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

In [ ]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())
    
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())